In [11]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

## S6 H2 adsorption isotherm, lumped injections

### Reading in the data and preparing DataFrames

In [12]:
#reading vaclogger measurement file
vaclog=pd.read_csv("vaclog2", sep="\t")
vaclog.head()

#adding an elapsed time column
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
0,NaN,2023-03-10 11:41:44,0.0,96.842,1.030000e-09,1.290000e-09,4.996000e-09,2.670000e-07,1.1,292.0,-5.050,4288.311,NaN,NaN
1,NaN,2023-03-10 11:41:51,7.0,96.846,1.030000e-09,1.290000e-09,4.996000e-09,2.673000e-07,1.1,292.0,-5.163,4288.984,NaN,NaN
2,NaN,2023-03-10 11:41:57,13.0,96.846,1.030000e-09,1.290000e-09,4.996000e-09,2.673000e-07,1.1,292.0,-4.938,4289.657,NaN,NaN
3,NaN,2023-03-10 11:42:04,20.0,96.847,1.030000e-09,1.280000e-09,4.996000e-09,2.676000e-07,1.1,292.0,-5.050,4290.555,NaN,NaN
4,NaN,2023-03-10 11:42:11,27.0,96.846,1.030000e-09,1.280000e-09,4.996000e-09,2.676000e-07,1.1,292.0,-5.163,4291.341,NaN,NaN


### Data processing

In [13]:
#print vaclog comments
print(pd.unique(vaclog["Live comments"]))

[nan 'recovered from serial hub error' 'starting first injection'
 'first injection complete' 'RGA filament on again'
 'restarted first ESD cycle' 'end of first esd cycle'
 'second injection complete' 'starting second esd cycle'
 'second esd cycle end' 'started third injection'
 'stopped third injection' 'LHe transfer' 'starting third esd cycle'
 'no visible esd' 'ending third esd cycle'
 'starting injection to reach sat vapor pressure' 'barion error']


#### CernOx temperature R-T conversion

In [14]:
#Temperature curve for CERNOX 
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


## Ignore end of measurement

In [15]:
#slicing the dataset
ramp = vaclog.iloc[list((vaclog["Elapsed time"]/3600).between(0,2))]
print(ramp)

     Live comments                Time  Elapsed time  injection 100mbar  \
0              NaN 2023-03-10 11:41:44           0.0             96.842   
1              NaN 2023-03-10 11:41:51           7.0             96.846   
2              NaN 2023-03-10 11:41:57          13.0             96.846   
3              NaN 2023-03-10 11:42:04          20.0             96.847   
4              NaN 2023-03-10 11:42:11          27.0             96.846   
...            ...                 ...           ...                ...   
1033           NaN 2023-03-10 13:41:13        7169.0             95.966   
1034           NaN 2023-03-10 13:41:20        7176.0             95.967   
1035           NaN 2023-03-10 13:41:27        7183.0             95.968   
1036           NaN 2023-03-10 13:41:34        7190.0             95.964   
1037           NaN 2023-03-10 13:41:41        7197.0             95.963   

          Barion_2      Barion_1  DUAL experiment  DUAL insulation  \
0     1.030000e-09  1.290000e

## Subtracting baseline

In [16]:
#TODO - calculating coverages for plotting pressure v coverage
#in the first assumption coverage is assumed to be equivalent to the number of molecules injected
#coverage at any given time point in [M/cm2] -> M/s * total injection time 
#First - Determine the inj start point, set elapsed time here to 0. 
inj = vaclog.loc[vaclog["Barion_2"]/0.46 >= 9e-8]
inj.drop(columns="Elapsed time", inplace=True)
print(inj)
timestamps2 = pd.to_datetime(inj["Time"],format="%d/%m/%Y %H:%M:%S")
runtime2 = (timestamps2-timestamps2[633]).dt.total_seconds()    
inj.insert(1,"Elapsed time",runtime2)



                                       Live comments                Time  \
56                                               NaN 2023-03-10 11:48:10   
57                                               NaN 2023-03-10 11:48:17   
58                                               NaN 2023-03-10 11:48:24   
59                                               NaN 2023-03-10 11:48:31   
60                                               NaN 2023-03-10 11:48:38   
...                                              ...                 ...   
3465                                             NaN 2023-03-10 18:22:40   
3466  starting injection to reach sat vapor pressure 2023-03-10 18:22:47   
3467                                             NaN 2023-03-10 18:22:54   
3468                                             NaN 2023-03-10 18:23:01   
3469                                             NaN 2023-03-10 18:23:08   

      injection 100mbar      Barion_2      Barion_1  DUAL experiment  \
56             

C:\Users\etiirine\AppData\Local\Temp\ipykernel_9672\522004775.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inj.drop(columns="Elapsed time", inplace=True)


KeyError: 633

In [ ]:
#TODO - subtract the baseline from graph 2 (linear pressure scale) and plot again
#adding H2 equiv pressure readings
inj.insert(3, "corr BA2", inj["Barion_2"]/0.46)
inj.insert(4, "corr BA1", inj["Barion_1"]/0.46)


### Thermal transpiration
Applied for the gauge readings using the formula below:
    $$
    \frac{p_2}{p_1}=\sqrt \frac{T_2}{T_1}
    $$
Here $p_2$ and $T_2$ are the pressure and temperature in the cold part and $p_1$, $T_1$ are the pressure, temperature the gauge is exposed to.

In [ ]:
#implementing the formula
T2 = 4.2
T1 = 293
p_coef = np.sqrt(T2/T1)
inj.insert(3, "H2 4K BA2", (inj["corr BA2"]*p_coef))
inj.insert(4, "H2 4K BA1", (inj["corr BA1"]*p_coef))


In [ ]:
print(p_coef)
inj.head()

In [ ]:
#define baseline
base1 = inj[inj["corr BA1"].between(4.6e-6,6e-6)]
base2 = inj[inj["corr BA2"].between(4.6e-6,6e-6)]

#take the average of the baseline and apply thermal transpiration coef to these values
avg2 = np.mean(base1["corr BA2"])*p_coef
avg1 = np.mean(base2["corr BA1"])*p_coef
print(avg1)
print(avg2)


In [ ]:
#subtract the baseline from the graph to get the actual pressure evolution on the sample

#Plotting the h2 equiv base pressure subtracted data
fig, ax = plt.subplots()
#subtract the baseline from the graph to get the actual pressure evolution on the sample

ax.plot(inj["Time"],inj['H2 4K BA1'].sub(avg1),marker=".", markersize=5,label='BA1')
ax.plot(inj["Time"],inj['H2 4K BA2'].sub(avg2),marker=".", markersize=5,label='BA2')
plt.legend(loc="upper left")

ax.set_xlabel('Timestamp')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.legend(loc="upper right")
plt.title("Pressure evolution on the sample, p increase from inj flux subtracted. \n Pressures H2 equivalent, thermal transp. corrected ")


### Injection pressure linearity

In [ ]:
#defining a function for the straight line
def func(x,a,b):
    return a*x + b
#curve fit for the data
y = inj["injection 100mbar"].values
x = (inj["Elapsed time"]/3600).values
params, cov = curve_fit(func,x,y)
#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.2f}*x+{b:.2f}")

### Formula for sum of the differential injection flux
$$
N=\frac{\Sigma dp\cdot V_{inj}}{k_B\cdot T}=\frac{\Sigma (p_{inj,t(x)}-p_{inj,t(x-1)})\cdot V_{inj}}{k_B \cdot T}   \space \left[{M}\right]
$$

In [ ]:
#constants in SI units
T = 293
k_B = 1.38e-23
V_inj = 6.515e-5
S_sample= 276
#implement the formula above
dp = abs(np.diff(inj["injection 100mbar"]*100))
print(np.mean(dp))
dp = np.insert(dp,0,0)
N_x = (dp.cumsum())*V_inj/(k_B*T)
print(N_x)
N_cov = (N_x/S_sample)
print(N_cov)


In [ ]:
inj.insert(2, "Coverage",N_cov)


In [ ]:
inj.head()

In [ ]:
## dropping coverage values close to 0
inj_new = inj.loc[inj['Coverage'] > 1e+13]
inj_new.head()
#inj_new["H2 4K BA2"].sub(avg2).describe()

# Plotting

In [ ]:
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 10,
        }

In [ ]:
#Plotting the general pressure evolution for overview
fig, ax = plt.subplots(figsize=(12,6))

l5 = ax.plot(vaclog["Time"], polyfit(A,vaclog["T-CERNOX"]),marker=".", color="cyan", markersize=5,label='Temperature')
ax.set_ylabel('Temperature (K)')
ax.set_xlabel('Timestamp')
ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
ax2=ax.twinx()
l1 = ax2.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
l2 = ax2.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
l3 = ax2.plot(vaclog["Time"],vaclog['DUAL experiment'],marker=".", markersize=4,label='Dual gauge')
l4 = ax2.plot(vaclog["Time"],vaclog['injection 100mbar'],marker=".", markersize=4,label='100mbar F-R')
ax2.set_ylabel('Pressure (mbar)')

plt.text(14.8, 7.36e-6, 'Pressure rise due to LHe refill \u2192 ', fontdict = font)
plt.text(21, 1e-5, '~Saturated vapor pressure', fontdict = font)
ax2.legend(handles = l1+l2+l3+l4+l5, loc="upper right")
plt.title("Pressure evolution + temperatures")
ax2.set_yscale('log')

plt.savefig(r'./graphs/S6 a-C H2 adsorption isotherm_overview.png')
plt.show()


In [ ]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.plot(vaclog["Time"],vaclog["DUAL experiment"],marker=".", markersize=4,label='DUAL experiment')
plt.plot(vaclog["Time"],vaclog['injection 100mbar'],marker=".", markersize=4,label='100mbar F-R')

plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("S1 CU ESD sample out - Total pressure evolution")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.savefig(r'./S6 a-C H2 adsorption isotherm_Pressures.png')
plt.show()

In [ ]:
#plotting the sliced data
#Plotting the general pressure evolution for overview
fig, ax = plt.subplots(figsize=(12,6))

ax.plot(ramp["Time"],ramp['Barion_1'],marker=".", markersize=4,label='BA1')
ax.plot(ramp["Time"],ramp['Barion_2'],marker=".", markersize=4,label='BA2')
ax.plot(ramp["Time"],ramp['DUAL experiment'],marker=".", markersize=4,label='DUAL gauge')

plt.legend(loc="upper right")

ax.set_xlabel('Timestamp')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')

plt.text(14.25, 6e-6, 'Pressure rise due to LHe refill \u2192 ', fontdict = font)
plt.text(13.02, 4.32e-6, 'Temperature fluctuation \u2192', fontdict = font)
plt.legend(loc="upper right")
plt.title("Pressure evolution during injection. Corrected for H2")

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.savefig(r'./S6 a-C H2 adsorption isotherm_Sliced df Pressures.png')
plt.show()


In [ ]:
#checking inj pressure linearity, graph
fig, ax = plt.subplots(figsize=(12,6))
y_fit = func(x,a,b)
ax.plot(inj["Time"],inj["injection 100mbar"],marker=".", markersize=5,label='inj volume pressure')
ax.legend(loc="upper right")

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
ax.set_xlabel('Timestamp')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')
plt.title("Injection pressure evolution 100mbar conductance gauge")

In [ ]:
#plot coverage vs time
fig, ax = plt.subplots(figsize=(12,6))

ax.plot(inj["Time"],inj["Coverage"],marker=".", markersize=5,label='Coverage')
ax.legend(loc="upper right")

ax.set_xlabel('Timestamp')
ax.set_ylabel('Coverage (M/s*cm2)')
ax.set_yscale('linear')
plt.title("Coverage over time")

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')   

plt.savefig(r'./S6 a-C H2 adsorption isotherm_Coverage vs time.png')
plt.show()


In [ ]:
#plotting pressure vs coverage
fig, ax = plt.subplots()

ax.plot((inj_new["Coverage"]),inj_new["H2 4K BA2"].sub(avg2),marker=".", markersize=5,label='isotherm')
ax.legend(loc="upper left")
xrange = np.arange(min(inj_new["Coverage"]),max(inj_new["Coverage"]),5e+15)
print(xrange)
ax.set_xticks(xrange)
#ax.set_xlim(left = 3e+15,right = 9.20e15)
#ax.set_xlim(right= 9.23e+15)
#ax.set_xlim(2.9e+12,9.23e+15)
#ax.set_ylim(1e-10,1.5e-6)


ax.set_xlabel('Coverage (M/cm2)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('log')
plt.title("H2 adsorption isotherm")

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')   

plt.savefig(r'./S6 a-C H2 adsorption isotherm.png')
plt.show()